$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\ketbra}[2]{\ket{#1}\bra{#2}}$$

# Import de Modules

In [ ]:
from qutip import *

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from scipy.fft import fft, fftfreq

plt.switch_backend("module://ipykernel.pylab.backend_inline")